# Let's build GPT: from scratch, in code, spelled out.

### :/ Andrej Karpathy Lecture::

In [18]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader

import numpy
import re
import unicodedata
import csv

In [19]:
with open('input.txt', 'r', encoding='utf-8') as file:
    txt = file.read()

In [3]:
print('Len of dataset in chars', len(txt))


Len of dataset in chars 1115394


In [4]:
print(txt[:200])
print('................')

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you
................


In [20]:
chars = sorted(list(set(txt)))
vocabSiz = len(chars)

print(''.join(chars))
print(vocabSiz)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


### encoder: take a str, output a list of int

### decoder: take a list of int, output a str

In [21]:
stoi = {ch:i for i, ch in enumerate(chars)}
itos = {i:ch for i, ch in enumerate(chars)}
enc = lambda s: [stoi[c] for c in s]
decod = lambda l: ''.join([itos[i] for i in l])

In [7]:
print(enc('hii there'))
print(decod(enc('hii thre')))

[46, 47, 47, 1, 58, 46, 43, 56, 43]
hii thre


### enc entire dataset and store in torch.Tensor

In [8]:
data = torch.tensor(enc(txt), dtype=torch.long)
print(data.shape, data.type)

print(data[:200])

torch.Size([1115394]) <built-in method type of Tensor object at 0x7520a030fba0>
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59])


## Split Data
 
 + 90% train, others dev/val 

In [22]:
n = int(0.9*len(data))
trainData = data[:n]
devData = data[n:]


In [10]:
# BlockSize/ ContextLength: max len of the char/token that could feed to trannsformer
blockSiz = 8
trainData[:blockSiz+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

## So the reason why [ y ] have skip sort of: [1:+1]
  
   + is, lets say ''x'' is, ['what is your name']

   + then ''y'' would be ['is your name']:

   *  cos is come after what, the model predicting the next token

++

#### So now if x [18, 47], y would predict [56], Cos its the next token and so forth 

In [23]:
x = trainData[:blockSiz]
y = trainData[1:blockSiz+1]

for token in range(blockSiz):
    contxt = x[:token+1]
    targt = y[token]

    print(f'when input is {contxt}, output is {targt} ')    

when input is tensor([18]), output is 47 
when input is tensor([18, 47]), output is 56 
when input is tensor([18, 47, 56]), output is 57 
when input is tensor([18, 47, 56, 57]), output is 58 
when input is tensor([18, 47, 56, 57, 58]), output is 1 
when input is tensor([18, 47, 56, 57, 58,  1]), output is 15 
when input is tensor([18, 47, 56, 57, 58,  1, 15]), output is 47 
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47]), output is 58 


# Batch Dim

 + Batch Size: 
    * how many indepndnt seq will be porcess in parallel
 
 + Block Size:
    * what is the max contxt len for prediction 

In [24]:
torch.manual_seed(1337)
bathSiz = 4
blockSiz = 8

def getBatch(split):
    data = trainData if split == 'train' else devData
    ix = torch.randint(len(data) - blockSiz, (bathSiz, ))
    x = torch.stack([data[i:i+blockSiz] for i in ix])
    y = torch.stack([data[i+1:i+blockSiz+1] for i in ix])

    return x, y

xb, yb = getBatch('train')
print('inputs')
print(xb.shape)
print(xb)

print('targts')
print(yb.shape)
print(yb)

print('\n.....')

for batch in range(bathSiz):
    for tokn in range(blockSiz):
        contxt = xb[batch, :tokn+1]
        targt = yb[batch, tokn]
        print(f'when input is {contxt}, output is {targt}')


inputs
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
targts
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])

.....
when input is tensor([24]), output is 43
when input is tensor([24, 43]), output is 58
when input is tensor([24, 43, 58]), output is 5
when input is tensor([24, 43, 58,  5]), output is 57
when input is tensor([24, 43, 58,  5, 57]), output is 1
when input is tensor([24, 43, 58,  5, 57,  1]), output is 46
when input is tensor([24, 43, 58,  5, 57,  1, 46]), output is 43
when input is tensor([24, 43, 58,  5, 57,  1, 46, 43]), output is 39
when input is tensor([44]), output is 53
when input is tensor([44, 53]), output is 56
when input is tensor([44, 53, 56]), output is 1
when input is tensor([44, 53, 5

In [13]:
# our input to transF
print(xb)

tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])


# Bigram Language Model,

 
   * ## Embedding:
     + so, emb layer will "plucks out" rows corresponding to token indices in your input.

     + Ex: ['whats your name'], emb will get conrespoding token for this input

# B T C, arragment:

  * Why This Arrangement?
     
     + The (B, T, C) shape allows models to handle mini-batches of sequences, where:
     
     + Each batch contains multiple sequences.
     
     + Each sequence has multiple tokens.
     
     + Each token is represented by an embedding vector.

  
### * Example Walkthrough
   
   + Suppose:

      - **Vocabulary size (vocab_size)**: 10 
      
      - **Embedding size (C)**: 4 (each token is mapped to a 4-dimensional vector)
      
      - **Batch size (B)**: 2 (you process 2 sequences at once)
      
      - **Sequence length (T)**: 3 (each sequence has 3 tokens)  


###### ----------
         ix = [
            [1, 3, 5],  # First sequence in the batch
            [0, 4, 2]   # Second sequence in the batch
         ]
###### ----------

  * This input tensor has the shape (2, 3):
      
      + B = 2: Two sequences (first and second).
      
      + T = 3: Three tokens in each sequence.

  
  *  Embedding Lookup   
   - The embedding layer "plucks out" embeddings for each token index:

         - For token 1 → [0.1, 0.2, 0.3, 0.4]
         - For token 3 → [0.5, 0.6, 0.7, 0.8]
         - For token 5 → [0.9, 1.0, 1.1, 1.2], etc.

### * The output is a (B, T, C) tensor:

 
            output = [
            [  # First sequence embeddings
               [0.1, 0.2, 0.3, 0.4],  # Embedding for token 1
               [0.5, 0.6, 0.7, 0.8],  # Embedding for token 3
               [0.9, 1.0, 1.1, 1.2]   # Embedding for token 5
            ],
            [  # Second sequence embeddings
               [0.01, 0.02, 0.03, 0.04],  # Embedding for token 0
               [0.5, 0.6, 0.7, 0.8],      # Embedding for token 4
               [0.9, 1.0, 1.1, 1.2]       # Embedding for token 2
            ]
            ]


### Shape: (2, 3, 4) where:

   + B=2 → 2 sequences.
   
   + T=3 → 3 tokens per sequence.
      
   + C=4 → Each token has a 4-dimensional embedding.


#### So in our case its [4, 8, 65]
  
  + where [ **2** ] is the seqLen or **Batch Size**

  + and [ **8** ] repr **Block Size**

  + lastly [ **65**]   is for, how many chars, vocab Size, 

## B T C ERROR:
  + however when we do this:


            return logits, loss
            logits, loss = model(xb, yb)  


  + we get error Cos,


      + RuntimeError: Expected target size [4, 65], got [4, 8]


  + Cos Torch expects B C T, not B T C like we have now , 
  its just the detail how torch treat these kind of inputs,
  so we not gonna deal with that 


  + instead what we gonna is **Reshape out Logits**
  


  

In [ ]:
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):
    def __init__(self, vocabSiz):
        super(BigramLanguageModel, self).__init__()
        self.tokenEmbTable = nn.Embedding(vocabSiz, vocabSiz) # 65x65
    
    def forward(self, ix, targt=None):
        #indx and target are both (B, T) tensor or int
        logits = self.tokenEmbTable(ix) # (B, T, C)
        
        if targt is None:
            loss = None
        
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targt = targt.view(B*T)
            loss = F.cross_entropy(logits, targt)

        return logits, loss

    def genarate(self, ix, maxNewTokn):
        #ix is (B,T) array of indeceis in currnt contxt
        
        for i in range(maxNewTokn):
            logits, loss = self(ix)

            # focus only on the last time step
            logits = logits[:, -1, :] #become (B, C)

            probs = F.softmax(logits, dim=1) # (B, C)

            # sample From the Distribution
            ixNext = torch.multinomial(probs, num_samples=1) # (B, 1)

            # append sample indx to the running seq 
            ix = torch.cat((ix, ixNext), dim=1) # (B, T+1)
        
        return ix
    
model = BigramLanguageModel(vocabSiz)
logits, loss = model(xb, yb)

print(logits.shape)
print(loss)



torch.Size([32, 65])


# _________________________________________________________

## Also we can guess what the NLLL Loss Should Be, 

  + Cos, 

         -ln(1/65) 
  
  
  + log or ln, l / 65 and negative of it is [ **4.17**]

  + But what we got is  [ **4.87**]


### * whats this mean is
  
  + The initial Predictions are not **Super Diffuse**, 
  
  + they've got Little Bit **Entrophy**
  and so we guessing Wrong


##### Diffuse Mean:
  + **how well** the model's **predictions are spread out or distributed** across the possible outcomes

# _________________________________________________________

In [ ]:
print(decod(model.genarate(ix=torch.zeros((1, 1), dtype=torch.long), maxNewTokn=100)[0].tolist()))
# abviously These Sampling are Garbage, so we would Train


Sr?qP-QWktXoL&jLDJgOLVz'RIoDqHdhsV&vLLxatjscMpwLERSPyao.qfzs$Ys$zF-w,;eEkzxjgCKFChs!iWW.ObzDnxA Ms$3


# Train...//

In [33]:
# Create Optimizer
optim = torch.optim.AdamW(model.parameters(), lr=1e-3)


/home/dkcode/miniconda3/envs/tfenv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [40]:
batchSiz = 32

for i in range(10000):
    xb, yb = getBatch('train')

    logits, loss = model(xb, yb)
    optim.zero_grad(set_to_none=True)
    loss.backward()
    optim.step()

print(loss.item())

2.453843355178833


In [ ]:
print(decod(model.genarate(ix=torch.zeros((1, 1), dtype=torch.long), maxNewTokn=100)[0].tolist()))
# Whoo, now we get some Less grabage shit


OTENRBESI RINous O:
Bupouly! ff lathe tat ter jur
ELYoraty,
I sper tornd be ho t helin, t masu kivin
